In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# import tensorflow as tf


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping
import mlflow
import mlflow.tensorflow
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
mlflow.tensorflow.autolog()


In [3]:
train_dir = "Dataset/Covid/train"
test_dir = "Dataset/Covid/test"


In [27]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    'Dataset/Covid/Train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_data = val_gen.flow_from_directory(
    'Dataset/Covid/Test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Found 2000 images belonging to 2 classes.
Found 328 images belonging to 2 classes.


In [28]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Drop 50% neurons
    layers.Dense(1, activation='sigmoid')
])

In [23]:
from tensorflow.keras.regularizers import l2

layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001))


<Dense name=dense_4, built=False>

In [6]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [13]:
import mlflow
import mlflow.tensorflow
import tensorflow as tf

mlflow.tensorflow.autolog()

with mlflow.start_run(run_name="COVID_CNN_Model"):

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    history = model.fit(
        train_data,
        validation_data=test_data,
        epochs=10
    )

    test_loss, test_acc = model.evaluate(test_data)
    mlflow.log_metric("test_loss", test_loss)
    mlflow.log_metric("test_accuracy", test_acc)

    model.save("covid_cnn_model.keras")  # local save

    # ✅ Fixed line
    mlflow.tensorflow.log_model(model, artifact_path="model")


2025/06/07 10:00:42 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.
2025/06/07 10:00:42 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9776 - loss: 0.0755

63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.9777 - loss: 0.0752 - val_accuracy: 0.9543 - val_loss: 0.0930
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.9865 - loss: 0.0406 - val_accuracy: 0.9543 - val_loss: 0.1526
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9870 - loss: 0.0386

63/63 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.9868 - loss: 0.0389 - val_accuracy: 0.9573 - val_loss: 0.0879
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9766 - loss: 0.0685

63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.9766 - loss: 0.0683 - val_accuracy: 0.9787 - val_loss: 0.0607
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.9847 - loss: 0.0392 - val_accuracy: 0.9756 - val_loss: 0.0807
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9852 - loss: 0.0379

63/63 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.9853 - loss: 0.0378 - val_accuracy: 0.9756 - val_loss: 0.0455
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.9899 - loss: 0.0293 - val_accuracy: 0.9787 - val_loss: 0.0690
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.9923 - loss: 0.0381 - val_accuracy: 0.9817 - val_loss: 0.0626
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.9893 - loss: 0.0394 - val_accuracy: 0.9787 - val_loss: 0.0793
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.9877 - loss: 0.0326 - val_accuracy: 0.9695 - val_loss: 0.0856


2025/06/07 10:14:15 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: '>=' not supported between instances of 'slice' and 'int'
2025/06/07 10:14:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/06/07 10:14:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 369ms/step - accuracy: 0.9848 - loss: 0.0463


2025/06/07 10:14:31 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/06/07 10:14:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [29]:
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [30]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

with mlflow.start_run(run_name="COVID_MobileNetV2"):

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=30,
        callbacks=[early_stop]
    )

    # Manual logging (optional)
    val_loss, val_acc = model.evaluate(val_data)
    mlflow.log_metric("val_loss", val_loss)
    mlflow.log_metric("val_accuracy", val_acc)

    # Save locally
    model.save("covid_mobilenetv2.keras")

    # Log to MLflow
    mlflow.tensorflow.log_model(model, artifact_path="model")


2025/06/07 10:49:52 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.
2025/06/07 10:49:52 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.
c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 983ms/step - accuracy: 0.6591 - loss: 0.6478

63/63 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.6608 - loss: 0.6451 - val_accuracy: 0.9421 - val_loss: 0.2090
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 990ms/step - accuracy: 0.9222 - loss: 0.2208

63/63 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.9223 - loss: 0.2205 - val_accuracy: 0.9268 - val_loss: 0.1769
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9468 - loss: 0.1569

63/63 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.9469 - loss: 0.1568 - val_accuracy: 0.9604 - val_loss: 0.1440
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9666 - loss: 0.1261

...

63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.9751 - loss: 0.0789 - val_accuracy: 0.9756 - val_loss: 0.0628
Epoch 16/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9742 - loss: 0.0729

63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.9741 - loss: 0.0729 - val_accuracy: 0.9787 - val_loss: 0.0585
Epoch 17/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 1082s 17s/step - accuracy: 0.9684 - loss: 0.0938 - val_accuracy: 0.9726 - val_loss: 0.0642
Epoch 18/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.9721 - loss: 0.0789 - val_accuracy: 0.9756 - val_loss: 0.0644
Epoch 19/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 914s 15s/step - accuracy: 0.9792 - loss: 0.0669 - val_accuracy: 0.9726 - val_loss: 0.0711


2025/06/07 12:09:05 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: '>=' not supported between instances of 'slice' and 'int'
2025/06/07 12:09:05 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/06/07 12:09:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 817ms/step - accuracy: 0.9763 - loss: 0.0583


2025/06/07 12:09:27 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/06/07 12:09:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [40]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved model
model = load_model("covid_cnn_model.keras")

# Load and preprocess a new image for prediction
img_path = "./Dataset/COVID/test/Covid Negative/person1_bacteria_1.jpeg"  # Replace with your image path
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # (1, 224, 224, 3)

# Predict
prediction = model.predict(img_array)[0][0]

# Interpret prediction
label = "Covid Positive" if prediction > 0.5 else "Covid Negative"
print(f"Prediction: {label} ({prediction:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Prediction: Covid Negative (0.00)


c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [45]:
from tensorflow.keras.models import load_model

model = load_model("covid_mobilenetv2.keras")

# Predict on a sample image
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img("./Dataset/COVID/test/Covid Positive/1-s2.0-S1341321X20301124-gr3_lrg-e.png", target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)[0][0]
print("Prediction:", "Covid Positive" if pred > 0.5 else "Covid Negative", f"({pred:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: Covid Positive (1.00)
